# benchmark spie sorter


MEArec is simulation tools....

spikeinterface have a comparison modules ...


Given the "templates_Neuronexus-32_100.h5" file we will generate some recording file.



In [3]:
%matplotlib widget

In [10]:
from pathlib import Path
import os

import numpy as np
import matplotlib.pyplot as plt

import MEArec as mr

import neo
import quantities as pq

import spikeinterface.full as si

## step 1 : generate recording

Here a small code that generate a recording given an already generated "templates" file.



In [11]:
basedir = Path('/home/samuel/Bureau/')
template_filename = basedir / 'templates_Neuronexus-32_100.h5'


duration = 30*60
n_cell = 10
probe = 'Neuronexus-32'
recording_filename = basedir / f'recordings_collision_{n_cell}cells_{probe}_{duration:0.0f}s.h5'

In [12]:
# some parameters
fs = 30000.
spikerate = 5. # Hz


In [13]:
# generate poisson spiketrains
spiketimes = []
for i in range(n_cell):
    n = int(spikerate * duration)
    stimes = np.random.rand(n) * duration
    stimes = np.sort(stimes)
    spiketimes.append(stimes)

# make neo spiketrains
spiketrains = []
for i in range(n_cell):
    spiketrain = neo.SpikeTrain(spiketimes[i], units='s', t_start=0*pq.s, t_stop=duration*pq.s)
    spiketrain.annotate(cell_type='E')
    spiketrains.append(spiketrain)

In [14]:
# generate signals from spiketrain
spgen = mr.SpikeTrainGenerator(spiketrains=spiketrains)

rec_params = mr.get_default_recordings_params()
rec_params['recordings']['fs'] = fs
rec_params['recordings']['sync_rate'] = None
rec_params['recordings']['sync_jitter'] = 5
rec_params['recordings']['noise_level'] = 5
rec_params['recordings']['filter'] = False
rec_params['recordings']['chunk_duration'] = 10.
rec_params['spiketrains']['duration'] = duration
rec_params['spiketrains']['n_exc'] = n_cell
rec_params['spiketrains']['n_inh'] = 0
rec_params['templates']['n_overlap_pairs'] = None
rec_params['templates']['min_dist'] = 0

recgen = mr.gen_recordings(params=rec_params, spgen=spgen, 
            templates=template_filename, verbose=True,
            n_jobs=1, tmp_mode='memmap')



Noise Level  5
Templates selection seed:  5657
Selecting cells
Padding template edges
Elapsed pad time: 0.19822382926940918
Elapsed resample time: 0.05106306076049805
Creating time jittering
Elapsed jitter time: 0.3166685104370117
Computing spike train SNR
Adding spiketrain annotations
Convolution seed:  1303
Electrode modulaton
Adding noise
Noise seed:  8435
Elapsed time:  78.15281399000014


In [15]:
mr.save_recording_generator(recgen, filename=recording_filename)

## step 2 : open and plot signals and spikes of ground truth

### step 3 : run several sorters on this

### step 4 : run comparison and plot agreement matrix

### step 5 use study object and plot performences